# Recommendation based on top 10 sales items (whole)

In [4]:
import pandas as pd
pd.options.mode.chained_assignment = None # hide SettingWithCopyWarning

df = pd.read_csv('D:/data_encoding_sel.csv')

In [6]:
def Rec_10(train,test,recall_num):
    # Calculate best 10 sell in training
    D = {}
    for i in train['mod_itemfullname']:
        D[i]= len(train[train['mod_itemfullname']==i]['mod_itemfullname'])

    # Sort dictionary in descending order    
    import operator
    sorted_d = dict( sorted(D.items(), key=operator.itemgetter(1),reverse=True))

    # Get first 10 hightest brought items
    from itertools import islice
    H = list(islice(sorted_d.items(), recall_num))
    print(f'Top {recall_num} highest brought (item,counts):\n',H)

    # Recommend list
    Rec_list = [i[0] for i in H]
    print('\nRecommend list:',Rec_list)

    # Recall@10
    rec10 = sum([1 for i in list(test['mod_itemfullname']) if i in Rec_list ])/len(list(test['mod_itemfullname']))
    print(f'Recall@{recall_num}:',rec10)

# 80 train, 20 test
train = df[0:4836]
test = df[4836:]    
Rec_10(train,test,5)

Top 5 highest brought (item,counts):
 [(537, 103), (465, 94), (659, 82), (330, 76), (597, 76)]

Recommend list: [537, 465, 659, 330, 597]
Recall@5: 0.04466501240694789


# Recommendation based on top 10 sales items (week)

In [42]:
from itertools import islice
import operator
df['user_id'] = pd.to_datetime(df['transaction_date']).apply(lambda time: time.dayofweek)

In [43]:

def Rec_week(train,test,recall_num):    
    test_actual = list(test['mod_itemfullname'])
    
    def Rec_list(idx,Day):
        M = train[train['user_id']==idx]
        D = {}
        for i in M['mod_itemfullname']:
            D[i]= len(M[M['mod_itemfullname']==i]['mod_itemfullname'])

        # Sort dictionary in descending order    
        sorted_d = dict( sorted(D.items(), key=operator.itemgetter(1),reverse=True))
        
        # Get first 10 hightest brought items
        H = list(islice(sorted_d.items(), len(sorted_d)))
        #print('Top 10 highest brought (item,counts):\n',H)

        # Recommend list
        Rec_list = [i[0] for i in H]
        print('{} recommended list:'.format(Day),Rec_list[0:recall_num])
        return Rec_list

    # Best 10 sales items for everday
    L = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    List_saved = {}
    for idx,Day in enumerate(L):
        List_saved[idx] = Rec_list(idx,Day)


    MRR_res = 0
    s = 0
    for i in range(len(test)):
        if test_actual[i] in List_saved[list(test['user_id'])[i]]:
            idx = List_saved[list(test['user_id'])[i]].index(test_actual[i])
            MRR_res += 1/(idx+1)    
        
            if test_actual[i] in List_saved[list(test['user_id'])[i]][0:recall_num]:
                s+=1
    print(s)
    print('\nMRR: ',MRR_res/len(test_actual))
    print(f'Recall@{recall_num}: ',s/len(test_actual))

# 80 train, 20 test
train = df[0:4836]
test = df[4836:]
Rec_week(train,test,3)

Monday recommended list: [537, 330, 464]
Tuesday recommended list: [537, 531, 659]
Wednesday recommended list: [465, 330, 407]
Thursday recommended list: [413, 407, 659]
Friday recommended list: [597, 354, 465]
Saturday recommended list: [597, 659, 465]
Sunday recommended list: [659, 440, 120]
42

MRR:  0.0458449439216369
Recall@3:  0.034739454094292806


# Recommendation based on top 10 sales items (time)

In [40]:
from datetime import datetime
df['user_id']= [0 for i in range(len(df))]

for i in range(len(df)):
    d_target = datetime.strptime(df['transaction_date'][i][11:], "%H:%M:%S")

    if datetime.strptime('08:00:00', "%H:%M:%S") <= d_target < datetime.strptime('11:00:00', "%H:%M:%S"):
        df['user_id'][i] = 0
    elif datetime.strptime('11:00:00', "%H:%M:%S") <= d_target < datetime.strptime('14:00:00', "%H:%M:%S"):
        df['user_id'][i] = 1 
    elif datetime.strptime('14:00:00', "%H:%M:%S") <= d_target < datetime.strptime('17:00:00', "%H:%M:%S"):
        df['user_id'][i] = 2 
    elif datetime.strptime('17:00:00', "%H:%M:%S") <= d_target < datetime.strptime('20:00:00', "%H:%M:%S"):
        df['user_id'][i] = 3 
    elif datetime.strptime('20:00:00', "%H:%M:%S") <= d_target < datetime.strptime('23:59:59', "%H:%M:%S"):
        df['user_id'][i] = 4
    elif datetime.strptime('00:00:00', "%H:%M:%S") <= d_target < datetime.strptime('08:00:00', "%H:%M:%S"):
        df['user_id'][i] = 5

In [41]:
def Rec_time(train,test,recall_num):    
    test_actual = list(test['mod_itemfullname'])
    
    def Rec_list(idx,Day):
        M = train[train['user_id']==idx]
        D = {}
        for i in M['mod_itemfullname']:
            D[i]= len(M[M['mod_itemfullname']==i]['mod_itemfullname'])

        # Sort dictionary in descending order    
        sorted_d = dict( sorted(D.items(), key=operator.itemgetter(1),reverse=True))

        # Get first 10 hightest brought items
        H = list(islice(sorted_d.items(), len(sorted_d)))
        #print('Top 10 highest brought (item,counts):\n',H)

        # Recommend list
        Rec_list = [i[0] for i in H]
        print('{} recommended list:'.format(Day),Rec_list[0:recall_num])
        return Rec_list

    # Best 10 sales items for time
    L = ['08.00-11.00','11.00-14.00','14.00-17.00','17.00-20.00','20.00-00.00','00.00-08.00']
    List_saved = {}
    for idx,Day in enumerate(L):
        List_saved[idx] = Rec_list(idx,Day)

    MRR_res = 0
    s = 0
    for i in range(len(test)):
        if test_actual[i] in List_saved[list(test['user_id'])[i]]:
            idx = List_saved[list(test['user_id'])[i]].index(test_actual[i])
            MRR_res += 1/(idx+1)    
        
            if test_actual[i] in List_saved[list(test['user_id'])[i]][0:recall_num]:
                s+=1

    print(s)
    print('\nMRR: ',MRR_res/len(test_actual))
    print(f'Recall@{recall_num}:',s/len(test_actual))

# 80 train, 20 test
train = df[0:4836]
test = df[4836:]
Rec_time(train,test,3)

08.00-11.00 recommended list: [531, 118, 306]
11.00-14.00 recommended list: [465, 464, 659]
14.00-17.00 recommended list: [330, 407, 597]
17.00-20.00 recommended list: [537, 659, 407]
20.00-00.00 recommended list: [199, 330, 169]
00.00-08.00 recommended list: [231, 413, 514]
74

MRR:  0.06790639505173728
Recall@3: 0.06120760959470637


In [17]:
def Rec_time(train,test,recall_num):    
    test_actual = list(test['mod_itemfullname'])
    
    def Rec_list(idx,Day):
        M = train[train['user_id']==idx]
        D = {}
        for i in M['mod_itemfullname']:
            D[i]= len(M[M['mod_itemfullname']==i]['mod_itemfullname'])

        # Sort dictionary in descending order    
        sorted_d = dict( sorted(D.items(), key=operator.itemgetter(1),reverse=True))

        # Get first 10 hightest brought items
        H = list(islice(sorted_d.items(), len(sorted_d)))
        #print('Top 10 highest brought (item,counts):\n',H)

        # Recommend list
        Rec_list = [i[0] for i in H]
        print('{} recommended list:'.format(Day),Rec_list[0:recall_num])
        return Rec_list

    # Best 10 sales items for time
    L = ['08.00-11.00','11.00-14.00','14.00-17.00','17.00-20.00','20.00-00.00','00.00-08.00']
    List_saved = {}
    for idx,Day in enumerate(L):
        List_saved[idx] = Rec_list(idx,Day)

    MRR_res = 0
    s = 0
    for i in range(len(test)):
        if test_actual[i] in List_saved[list(test['user_id'])[i]][0:recall_num]:
            s+=1
        elif test_actual[i] in List_saved[list(test['user_id'])[i]]:
            idx = List_saved[list(test['user_id'])[i]].index(test_actual[i])
            MRR_res += 1/(idx+1)

    print(s)
    print('\nMRR: ',MRR_res/len(test_actual))
    print(f'Recall@{recall_num}:',s/len(test_actual))

# 80 train, 20 test
train = df[0:4836]
test = df[4836:]
Rec_time(train,test,3)

08.00-11.00 recommended list: [531, 118, 306]
11.00-14.00 recommended list: [465, 464, 659]
14.00-17.00 recommended list: [330, 407, 597]
17.00-20.00 recommended list: [537, 659, 407]
20.00-00.00 recommended list: [199, 330, 169]
00.00-08.00 recommended list: [231, 413, 514]
74

MRR:  0.02737703194172884
Recall@3: 0.06120760959470637


# Recommendation based on top 10 sales items (week and time)

In [37]:
df['user_id1'] = pd.to_datetime(df['transaction_date']).apply(lambda time: time.dayofweek)
df['user_id'] = [0 for i in range(len(df))]

In [38]:
for i in range(len(df['transaction_date'])):
    d_target = datetime.strptime(df['transaction_date'][i][11:], "%H:%M:%S")
    
    for j in range(7):
        if df['user_id1'][i]==j:  # Monday
            if datetime.strptime('08:00:00', "%H:%M:%S") <= d_target < datetime.strptime('11:00:00', "%H:%M:%S"):
                df['user_id'][i] = j*6
            elif datetime.strptime('11:00:00', "%H:%M:%S") <= d_target < datetime.strptime('14:00:00', "%H:%M:%S"):
                df['user_id'][i] = j*6+1 
            elif datetime.strptime('14:00:00', "%H:%M:%S") <= d_target < datetime.strptime('17:00:00', "%H:%M:%S"):
                df['user_id'][i] = j*6+2  
            elif datetime.strptime('17:00:00', "%H:%M:%S") <= d_target < datetime.strptime('20:00:00', "%H:%M:%S"):
                df['user_id'][i] = j*6+3  
            elif datetime.strptime('20:00:00', "%H:%M:%S") <= d_target < datetime.strptime('23:59:59', "%H:%M:%S"):
                df['user_id'][i] = j*6+4 
            elif datetime.strptime('00:00:00', "%H:%M:%S") <= d_target < datetime.strptime('08:00:00', "%H:%M:%S"):
                df['user_id'][i] = j*6+5

In [39]:
def Rec_weektime(train,test,recall_num):    
    test_actual = list(test['mod_itemfullname'])
    
    def Rec_list(idx,time):
        
        M = train[train['user_id']==idx]
        D = {}
        for i in M['mod_itemfullname']:
            D[i]= len(M[M['mod_itemfullname']==i]['mod_itemfullname'])

        # Sort dictionary in descending order    
        sorted_d = dict( sorted(D.items(), key=operator.itemgetter(1),reverse=True))

        # Get first 10 hightest brought items
        H = list(islice(sorted_d.items(), len(sorted_d)))
        #print('Top 10 highest brought (item,counts):\n',H)

        # Recommend list
        Rec_list = [i[0] for i in H]
        print('{} recommended list:'.format(time),Rec_list[0:recall_num])
        return Rec_list

    T = [q for q in range(42)] 
    
    List_saved = {}
    for idx,time in enumerate(T):
        List_saved[idx] = Rec_list(idx,time)

    MRR_res = 0
    s = 0
    for i in range(len(test)):
        if test_actual[i] in List_saved[list(test['user_id'])[i]]:
            idx = List_saved[list(test['user_id'])[i]].index(test_actual[i])
            MRR_res += 1/(idx+1)    
        
            if test_actual[i] in List_saved[list(test['user_id'])[i]][0:recall_num]:
                s+=1
    print(s)
    print('\nMRR: ',MRR_res/len(test_actual))
    print(f'Recall@{recall_num}:',s/len(test_actual))


# 80 train, 20 test
train = df[0:4836]
test = df[4836:]
Rec_weektime(train,test,3)

0 recommended list: [531, 306, 75]
1 recommended list: [464, 465, 466]
2 recommended list: [304, 330, 306]
3 recommended list: [537, 407, 145]
4 recommended list: [174, 456, 38]
5 recommended list: [231, 693, 413]
6 recommended list: [531, 306, 118]
7 recommended list: [537, 465, 659]
8 recommended list: [537, 145, 134]
9 recommended list: [537, 330, 531]
10 recommended list: [513, 666, 200]
11 recommended list: [413, 231, 304]
12 recommended list: [306, 531, 118]
13 recommended list: [465, 466, 309]
14 recommended list: [145, 111, 694]
15 recommended list: [407, 121, 664]
16 recommended list: [330, 114, 102]
17 recommended list: [330, 306, 413]
18 recommended list: [531, 533, 304]
19 recommended list: [413, 420, 465]
20 recommended list: [507, 407, 413]
21 recommended list: [659, 146, 290]
22 recommended list: [169, 659, 684]
23 recommended list: [306, 304, 231]
24 recommended list: [531, 118, 120]
25 recommended list: [354, 465, 464]
26 recommended list: [597, 407, 134]
27 recommende